In [1]:
import pandas as pd

df = pd.read_csv('violations.csv')

In [2]:
df

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,4714702166,KGK6659,NY,PAS,11/12/2020,36,4DSD,BMW,V,0,...,GY,NaN,2007,NaN,0,NaN,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN
1,8793684599,L5232HY,TN,PAS,09/14/2020,21,TRAC,INTER,T,60790,...,WHITE,NaN,0,NaN,0,W,NaN,NaN,NaN,NaN
2,8864757053,BPMN76,FL,PAS,11/25/2020,20,SUBN,NISSA,T,36030,...,BLACK,NaN,0,NaN,0,49,20A-No Parking (Non-COM),NaN,NaN,NaN
3,8778326047,57387MJ,NY,COM,08/20/2020,38,VAN,NaN,T,34430,...,WH,NaN,2016,NaN,0,01,NaN,NaN,NaN,NaN
4,4706640702,M81KFJ,NJ,PAS,09/09/2020,36,UT,TOYOT,V,0,...,NaN,NaN,2002,NaN,0,NaN,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4014245755,JPN4910,NY,PAS,06/25/2020,5,PICK,FORD,V,0,...,RD,NaN,2019,NaN,0,NaN,BUS LANE VIOLATION,NaN,NaN,NaN
49996,8794437506,174ZYG,CT,PAS,08/25/2020,31,4DSD,TOYOT,T,34030,...,BLACK,NaN,0,NaN,0,16,NaN,NaN,NaN,NaN
49997,8849769581,HES6895,NY,PAS,11/03/2020,46,SUBN,CADIL,T,27120,...,RD,NaN,2005,NaN,0,K,NaN,NaN,NaN,NaN
49998,8755448173,63035MN,NY,COM,08/19/2020,40,DELV,ISUZU,T,0,...,WH,NaN,2020,NaN,5,67,NaN,NaN,NaN,NaN
